In [1]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, confidence_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.nearest_neighbours import CosineRecommender
from implicit.evaluation import train_test_split, ranking_metrics_at_k

In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
steam_df = (
    pd.read_csv(
        "/home/coder/projects/rec-sys-research/data/steam/steam_recommendations.csv",
        usecols=['user_id', 'app_id', 'hours'],
    )
    .loc[:, ['user_id', 'app_id', 'hours']]
    .drop_duplicates()
    .dropna()
    .rename(columns={'app_id': 'item_id', 'hours': 'target'})
)
steam_df['user_id'].nunique(), steam_df['item_id'].nunique(), steam_df.shape[0]

(13781059, 37610, 41154773)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    steam_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (13781059, 37610), Val Shape: (13781059, 37610), Test Shape: (13781059, 37610)


In [5]:
results_folder = "results/steam_knn"
results_filename = "steam_knn_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "KNN_k=20": lambda: CosineRecommender(K=20),
        "KNN_k=100": lambda: CosineRecommender(K=100),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "confidence",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "confidence":
            weighted = confidence_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "confidence":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            # params["p"] = trial.suggest_float("p", 0.1, 1.5)
            params["p"] = trial.suggest_float("p", 0.2, 1.3)
        weighted_train = get_weighted_matrix(train_mat, params)

        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ranking_metrics_at_k(model, train_mat, val_mat, K=20, show_progress=False)['ndcg']

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "confidence", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    metrics_at_10 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    metrics_at_20 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": metrics_at_10['ndcg'],
        "Test NDCG@20": metrics_at_20['ndcg'],
        "Test Precision@10": metrics_at_10['precision'],
        "Test Precision@20": metrics_at_20['precision'],
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [6]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [7]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 10:29:27,025] A new study created in memory with name: no-name-3ccf8bb3-3610-4824-9b6a-7a71ce012cdc


Running optimization for KNN_k=20 with no_weighting...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.30255556106567383 seconds
  warnings.warn(
[I 2026-02-10 10:32:28,695] Trial 0 finished with value: 0.03317366163332138 and parameters: {}. Best is trial 0 with value: 0.03317366163332138.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3275165557861328 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,no_weighting,1,0.033174,0.032947,0.039046,0.054574,0.077333,2.659427,{}


In [9]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 10:49:53,482] A new study created in memory with name: no-name-3033a303-0c90-4d03-bb95-6cd6dc2bc543


Running optimization for KNN_k=20 with bm25...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3197517395019531 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.31922006607055664 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.4640777111053467 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:1

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,bm25,10,0.050579,0.046935,0.056762,0.080643,0.11796,2.696452,"{'bm25_k1': 31.33329274964659, 'bm25_b': 0.777..."


In [10]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)


[I 2026-02-10 11:38:24,912] A new study created in memory with name: no-name-234d60e3-a3ac-4bd8-8e89-cb74e5dc8efd


Running optimization for KNN_k=20 with tfidf...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.2981078624725342 seconds
  warnings.warn(
[I 2026-02-10 11:41:27,373] Trial 0 finished with value: 0.04625727221801684 and parameters: {}. Best is trial 0 with value: 0.04625727221801684.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3280060291290283 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,tfidf,1,0.046257,0.043153,0.051849,0.074355,0.107704,2.658919,{}


In [11]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 11:48:03,721] A new study created in memory with name: no-name-7ba4244d-6181-4d03-b570-6aa124fb98b4


Running optimization for KNN_k=20 with log...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.29839444160461426 seconds
  warnings.warn(
[I 2026-02-10 11:51:06,241] Trial 0 finished with value: 0.049217863420257676 and parameters: {}. Best is trial 0 with value: 0.049217863420257676.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.32919907569885254 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,log,1,0.049218,0.045598,0.054935,0.078724,0.114667,2.669255,{}


In [12]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 11:57:42,026] A new study created in memory with name: no-name-a19b1076-e555-4baf-bb0d-8948f9a7951e


Running optimization for KNN_k=20 with confidence...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3242452144622803 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.32161664962768555 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3671422004699707 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.32570481300354004 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packag

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,confidence,10,0.051668,0.045684,0.055217,0.078853,0.115229,2.677147,{'conf_alpha': 39.30074639498613}


In [13]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 12:46:16,766] A new study created in memory with name: no-name-413144d9-68df-4009-a3f4-bf772178671a


Running optimization for KNN_k=20 with power...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3825984001159668 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.33550596237182617 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.4197831153869629 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:1

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,power,10,0.049771,0.045984,0.055527,0.079498,0.116189,2.734898,{'power_p': 0.16643832838443454}


In [14]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 13:34:13,588] A new study created in memory with name: no-name-2323d629-e021-445c-b2b8-2954201a0dbc


Running optimization for KNN_k=20 with normalized...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:39: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3251636028289795 seconds
  warnings.warn(
[I 2026-02-10 13:37:17,122] Trial 0 finished with value: 0.0021026602927233505 and parameters: {}. Best is trial 0 with value: 0.0021026602927233505.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3462481498718261

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,normalized,1,0.002103,0.003591,0.002727,0.006848,0.005739,2.839982,{}


In [15]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 13:44:24,953] A new study created in memory with name: no-name-85674416-7b46-42be-9584-b6975e6c0fd3


Running optimization for KNN_k=20 with pmi...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:155: RuntimeWarning: divide by zero encountered in log
  pmi = log((X.data * N) / denominator) # we could use np.power(X.data, p) instead of log for a softer effect
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.31746482849121094 seconds
  warnings.warn(
[I 2026-02-10 13:47:29,275] Trial 0 finished with value: 0.04768733965605798 and parameters: {}. Best is trial 0 with value: 0.04768733965605798.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix inst

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,pmi,1,0.047687,0.044153,0.053316,0.076539,0.111868,2.815338,{}


In [16]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 13:54:24,935] A new study created in memory with name: no-name-5a6729fb-17c2-437e-b7fe-5b358abb4c67


Running optimization for KNN_k=20 with robust_user_centric...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:294: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3222200870513916 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.32319188117980957 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.323108434677124 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,robust_user_centric,10,0.051041,0.045036,0.054587,0.077673,0.11412,2.819693,{'scale_factor': 9.607177688936702}


In [11]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-11 14:06:06,057] A new study created in memory with name: no-name-936ecf02-46ee-4bf8-9fd0-35eadb1a9869


Running optimization for KNN_k=20 with robust_user_centric_weight_v2...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:364: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.345059871673584 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3586452007293701 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.350538969039917 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, 

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,robust_user_centric_weight_v2,10,0.051632,0.045473,0.055058,0.07842,0.115005,2.973098,"{'lower_q': 16.77507398129061, 'upper_q': 94.5..."


In [8]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 17:31:23,624] A new study created in memory with name: no-name-1fac9482-1d15-43c4-b4fc-8bc7d65f4a88


Running optimization for KNN_k=20 with sigmoid_propensity...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:390: RuntimeWarning: divide by zero encountered in log
  sum_log += np.sum(np.log(X.data[start:end]))
/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:404: RuntimeWarning: divide by zero encountered in power
  term = C * np.power(data_chunk, neg_p)
/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:404: RuntimeWarning: invalid value encountered in multiply
  term = C * np.power(data_chunk, neg_p)
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.33519601821899414 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3328230381011963 seconds
  warnings.

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,sigmoid_propensity,10,0.000108,0.000054,0.000097,0.000154,0.000373,2.740127,"{'p': 4.571656888585181, 'beta': 0.46188283140..."


In [7]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="KNN_k=20", n_trials=10, output_dir=results_folder)

[I 2026-02-10 16:43:50,995] A new study created in memory with name: no-name-82ebfcba-9765-4f92-b654-f4b8201ab252


Running optimization for KNN_k=20 with power_lift...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.32915186882019043 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.339099645614624 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.47663021087646484 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:1

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,power_lift,10,0.051308,0.047369,0.057228,0.08163,0.119258,2.755537,{'p': 0.24434787681681283}


In [8]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 10:39:03,658] A new study created in memory with name: no-name-581a908b-dcb0-4101-9033-0e43418fc7e0


Running optimization for KNN_k=100 with no_weighting...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.30565929412841797 seconds
  warnings.warn(
[I 2026-02-10 10:42:26,124] Trial 0 finished with value: 0.03503657093469969 and parameters: {}. Best is trial 0 with value: 0.03503657093469969.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3281428813934326 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,no_weighting,1,0.035037,0.032108,0.038481,0.054769,0.078682,2.847643,{}


In [8]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 21:21:40,768] A new study created in memory with name: no-name-03fe3213-4a95-4850-8d1b-ddf25ec14e51


Running optimization for KNN_k=100 with bm25...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.31780266761779785 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.31972837448120117 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3227853775024414 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,bm25,10,0.052653,0.044927,0.055103,0.079599,0.118547,2.879537,"{'bm25_k1': 47.35533899208951, 'bm25_b': 0.324..."


In [9]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)


[I 2026-02-10 22:13:42,207] A new study created in memory with name: no-name-8b165c34-699f-4fb4-b3f0-52cd1b006347


Running optimization for KNN_k=100 with tfidf...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3002908229827881 seconds
  warnings.warn(
[I 2026-02-10 22:17:01,386] Trial 0 finished with value: 0.04800290142885423 and parameters: {}. Best is trial 0 with value: 0.04800290142885423.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.33144378662109375 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,tfidf,1,0.048003,0.041996,0.051024,0.074192,0.1089,2.881835,{}


In [9]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 18:17:00,030] A new study created in memory with name: no-name-80569e20-ac51-4f47-ba4b-860e44f8a6cf


Running optimization for KNN_k=100 with log...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3181190490722656 seconds
  warnings.warn(
[I 2026-02-10 18:20:22,573] Trial 0 finished with value: 0.051434579943995674 and parameters: {}. Best is trial 0 with value: 0.051434579943995674.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3438248634338379 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,log,1,0.051435,0.044641,0.054344,0.078939,0.116149,2.953212,{}


In [10]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-11 13:08:32,883] A new study created in memory with name: no-name-186c57b4-b751-4d68-b4a0-7955467ed00d


Running optimization for KNN_k=100 with confidence...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.36251139640808105 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.39403438568115234 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.40358996391296387 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.49159836769104004 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-pack

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,confidence,10,0.054884,0.048238,0.058631,0.084611,0.124065,3.178591,{'conf_alpha': 54.954464354080116}


In [10]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 18:27:35,198] A new study created in memory with name: no-name-3003f95b-4ae4-4967-bd7f-b1bd25f61414


Running optimization for KNN_k=100 with power...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3249244689941406 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3213233947753906 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3250701427459717 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:16

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,power,10,0.050971,0.044707,0.054421,0.07919,0.116518,2.847769,{'power_p': 0.2540575635986686}


In [7]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-11 16:58:12,679] A new study created in memory with name: no-name-9adca430-8533-48e4-97e2-64202db67bff


Running optimization for KNN_k=100 with normalized...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:39: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3304581642150879 seconds
  warnings.warn(
[I 2026-02-11 17:01:43,512] Trial 0 finished with value: 0.00027654626799082535 and parameters: {}. Best is trial 0 with value: 0.00027654626799082535.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.37161946296691

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,normalized,1,0.000277,0.000488,0.000421,0.001064,0.001159,3.118231,{}


In [11]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 19:19:50,225] A new study created in memory with name: no-name-f7783399-91ca-476e-b02c-b29dcfa0f876


Running optimization for KNN_k=100 with pmi...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:155: RuntimeWarning: divide by zero encountered in log
  pmi = log((X.data * N) / denominator) # we could use np.power(X.data, p) instead of log for a softer effect
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.29875636100769043 seconds
  warnings.warn(
[I 2026-02-10 19:23:12,660] Trial 0 finished with value: 0.05020722359980607 and parameters: {}. Best is trial 0 with value: 0.05020722359980607.
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix inst

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,pmi,1,0.050207,0.043481,0.053079,0.077446,0.114489,2.848038,{}


In [10]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-10 22:24:09,382] A new study created in memory with name: no-name-d7776091-6395-4daa-9707-d2460f80de0b


Running optimization for KNN_k=100 with robust_user_centric...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:294: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3333003520965576 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.314777135848999 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3956339359283447 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input,

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,robust_user_centric,10,0.054129,0.047525,0.057806,0.083225,0.122233,2.904375,{'scale_factor': 1.2109369175291178}


In [6]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-11 16:00:13,646] A new study created in memory with name: no-name-cebd0d43-7290-4a1d-b612-abf520d693d4


Running optimization for KNN_k=100 with robust_user_centric_weight_v2...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:364: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3530311584472656 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.5096642971038818 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3576195240020752 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,robust_user_centric_weight_v2,10,0.0547,0.04793,0.058268,0.083896,0.123102,3.143589,"{'lower_q': 14.726291257870932, 'upper_q': 92...."


In [7]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-11 12:11:36,473] A new study created in memory with name: no-name-76b6f171-aca5-4c6a-8638-b8fb332b1a73


Running optimization for KNN_k=100 with sigmoid_propensity...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:390: RuntimeWarning: divide by zero encountered in log
  sum_log += np.sum(np.log(X.data[start:end]))
/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:404: RuntimeWarning: divide by zero encountered in power
  term = C * np.power(data_chunk, neg_p)
/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:404: RuntimeWarning: invalid value encountered in multiply
  term = C * np.power(data_chunk, neg_p)
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.37392377853393555 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.33895277976989746 seconds
  warnings

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,sigmoid_propensity,10,0.000074,0.000037,0.000063,0.000101,0.000228,3.070615,"{'p': 3.153823758518224, 'beta': 0.76724505961..."


In [6]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="KNN_k=100", n_trials=10, output_dir=results_folder)

[I 2026-02-11 11:13:54,842] A new study created in memory with name: no-name-21d3860a-4a05-4229-89b8-a6835adfb7aa


Running optimization for KNN_k=100 with power_lift...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.3633842468261719 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.36844635009765625 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.34619879722595215 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,power_lift,10,0.053267,0.045968,0.056272,0.081386,0.120801,3.157452,{'p': 0.29080951981015246}


In [9]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,confidence,10,0.054884,0.048238,0.058631,0.084611,0.124065,3.178591,{'conf_alpha': 54.954464354080116}
0,KNN_k=100,robust_user_centric_weight_v2,10,0.054700,0.047930,0.058268,0.083896,0.123102,3.143589,"{'lower_q': 14.726291257870932, 'upper_q': 92...."
0,KNN_k=100,robust_user_centric,10,0.054129,0.047525,0.057806,0.083225,0.122233,2.904375,{'scale_factor': 1.2109369175291178}
0,KNN_k=20,power_lift,10,0.051308,0.047369,0.057228,0.081630,0.119258,2.755537,{'p': 0.24434787681681283}
0,KNN_k=20,bm25,10,0.050579,0.046935,0.056762,0.080643,0.117960,2.696452,"{'bm25_k1': 31.33329274964659, 'bm25_b': 0.777..."
0,KNN_k=100,power_lift,10,0.053807,0.045878,0.056078,0.081257,0.120402,3.230722,{'p': 0.259185148987678}
0,KNN_k=20,power,10,0.049771,0.045984,0.055527,0.079498,0.116189,2.734898,{'power_p': 0.16643832838443454}
0,KNN_k=20,confidence,10,0.051668,0.045684,0.055217,0.078853,0.115229,2.677147,{'conf_alpha': 39.30074639498613}
0,KNN_k=100,bm25,10,0.052653,0.044927,0.055103,0.079599,0.118547,2.879537,"{'bm25_k1': 47.35533899208951, 'bm25_b': 0.324..."
0,KNN_k=20,robust_user_centric_weight_v2,10,0.051632,0.045473,0.055058,0.078420,0.115005,2.973098,"{'lower_q': 16.77507398129061, 'upper_q': 94.5..."
